<a href="https://colab.research.google.com/github/Sudoyan118/llmatch_kadai/blob/main/LLMATCH_kadai_Gemma3_ver2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## はじめに

本ノートブックでは、Googleが提供するマルチモーダルな指示チューニング済みモデル google/gemma-3-4b-it を対象に、以下の2つのタスクに取り組むことを目的とした。


**タスク1:事前学習済みモデルの評価**

google/gemma-3-4b-it を以下の日本語VLM評価用ベンチマークにおいて評価する：
- Heron-Bench
- JA-VLM-Bench-In-the-Wild
- JA-VG-VQA-500

**タスク2:ファインチューニングと再評価**

日本語ビジュアル質問応答データセット line-corporation/JIC-VQA を用いて google/gemma-3-4b-it をファインチューニングし、上記3つのベンチマークを用いて再度評価を行う。</br></br>
しかし、最終的には全てのタスクを完了することはできなかった。
現時点で実施できた内容は以下の通りとなっている：
* google/gemma-3-4b-itをJA-VLM-Bench-In-the-WildとJA-VG-VQA-500のベンチマークを使って、Rouge-Lで評価する
* line-corporation/JIC-VQA · Datasets at Hugging Faceでgoogle/gemma-3-4b-itをファインチューニングしたのち、JA-VLM-Bench-In-the-WildとJA-VG-VQA-500のベンチマークを使って、Rouge-Lで再評価する


※ 本プロジェクトでは、Googleが提供する指示チューニング済みマルチモーダルモデル [`google/gemma-3-4b-it`](https://huggingface.co/google/gemma-3-4b-it) を使用しています。 また、本モデルの使用にあたっては、[Gemma RAIL License](https://huggingface.co/google/gemma-3-4b-it/blob/main/RAIL-LICENSE.txt) に準拠しています。




## ライブラリのインストールとバージョン確認

In [1]:
!pip install -q transformers accelerate datasets peft bitsandbytes gcsfs evaluate rouge_score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 83.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   

In [2]:
!pip install -U bitsandbytes

In [3]:
# バージョン確認したいライブラリ名
target_modules = [
    "transformers",
    "torch",
    "datasets",
    "accelerate",
    "peft",
    "bitsandbytes",
    "fsspec",
    "gcsfs",
    "evaluate",
    "rouge_score"
]

print("=== ライブラリバージョン一覧 ===")

for module_name in target_modules:
    try:
        mod = __import__(module_name)
        print(f"{module_name}: {mod.__version__}")
    except ImportError:
        print(f"{module_name}: インストールされていません")
    except AttributeError:
        print(f"{module_name}: `__version__` が存在しません")



=== ライブラリバージョン一覧 ===
transformers: 4.51.3
torch: 2.6.0+cu124
datasets: 3.5.0
accelerate: 1.5.2
peft: 0.14.0
bitsandbytes: 0.45.5
fsspec: 2024.12.0
gcsfs: 2024.12.0
evaluate: 0.4.3
rouge_score: `__version__` が存在しません


version属性で取得できないライブラリに対して個別でpip showコマンドを用いて確認する

In [4]:
!pip show rouge_score

Name: rouge_score
Version: 0.1.2
Summary: Pure python implementation of ROUGE-1.5.5.
Home-page: https://github.com/google-research/google-research/tree/master/rouge
Author: Google LLC
Author-email: rouge-opensource@google.com
License: 
Location: /usr/local/lib/python3.11/dist-packages
Requires: absl-py, nltk, numpy, six
Required-by: 


## Hugging Faceのアクセストークンログイン

In [5]:
from huggingface_hub import login
from google.colab import userdata
login(userdata.get("HF_token"))

## ライブラリのインポート



In [6]:
import torch
from transformers import (
    AutoProcessor, AutoTokenizer, AutoModelForImageTextToText,
    Gemma3ForConditionalGeneration, BitsAndBytesConfig,
    Trainer, TrainingArguments, default_data_collator
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import load_dataset
from PIL import Image
from io import BytesIO
import requests
import evaluate

## メモリの使用量の出力

In [7]:
def print_gpu_memory(stage=""):
    if torch.cuda.is_available():
        print(f"[{stage}] GPU Memory Usage:")
        print(f"  Allocated: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
        print(f"  Reserved : {torch.cuda.memory_reserved() / 1024**3:.2f} GB")

## 量子化の設定とモデル・プロセッサの初期化

* Gemma-3-4b-it：Googleが公開したInstruct-tunedなモデル。指示文に従った生成が得意。

* BitsAndBytesConfig：推論メモリを削減するための 4bit量子化の設定。torch_dtype=bfloat16 によって高速演算。

* AutoProcessor：画像とテキストの同時処理に必要な前処理器。

In [8]:
# ---------- 量子化の設定 ----------
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)

# ---------- モデル・プロセッサの初期化 ----------
gemma_model_id = "google/gemma-3-4b-it"
gemma_model = Gemma3ForConditionalGeneration.from_pretrained(
    gemma_model_id,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.bfloat16
).eval()
gemma_processor = AutoProcessor.from_pretrained(gemma_model_id)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/90.6k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.64G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

In [9]:
print_gpu_memory("after Initialize model")

[after Initialize model] GPU Memory Usage:
  Allocated: 3.01 GB
  Reserved : 4.75 GB


## ベンチマーク評価

## VQA推論関数
* テキストと画像をプロンプト形式にまとめて generate に渡す。

* 推論で得られた出力から余計なトークンを除去して pred_answer として返す。

In [10]:
def generate_vqa_answer(image, question, gt_answer=None):
    try:
        prompt = (
            f"<start_of_image><end_of_image>\n"
            f"この画像について「{question.strip()}」という質問に対して、簡潔に日本語で答えてください。"
        )
        inputs = gemma_processor(
            text=prompt,
            images=image,
            return_tensors="pt"
        )
        inputs = {k: v.to(gemma_model.device) for k, v in inputs.items()}

        with torch.inference_mode():
            outputs = gemma_model.generate(
                **inputs,
                max_new_tokens=100,
                do_sample=False
            )

        pred_answer = gemma_processor.decode(outputs[0], skip_special_tokens=True)
        pred_answer = pred_answer.strip().split("\n")[-1].strip()

    except Exception as e:
        print(f" Error during inference: {e}")
        pred_answer = "ERROR"

    return {
        "question": question,
        "gt_answer": gt_answer,
        "pred_answer": pred_answer.strip()
    }

## 推論ルーチン (JA-VG-VQA-500)
* JA-VG-VQA-500：1画像に複数質問


In [11]:
def infer_ja_vg_vqa500(dataset, model, processor, image_field="image", qa_field="qas"):
    results = []

    for img_idx, item in enumerate(dataset):
        image = item[image_field].convert("RGB")

        print(f"\n====== [VQA] Image {img_idx} の質問処理 ======")

        for qa_idx, qa in enumerate(item[qa_field]):
            question = qa["question"]
            gt_answer = qa.get("answer", None)

            result = generate_vqa_answer(image, question, gt_answer)
            results.append(result)

            print(f"[Image {img_idx} | QA {qa_idx}] 質問: {question}")
            print(f"           正解      : {gt_answer}")
            print(f"           生成回答  : {result['pred_answer']}\n")

    return results

## 推論ルーチン (JA-VLM-Bench-In-the-Wild)
* JA-VLM-Bench-In-the-Wild：1画像1質問



In [12]:
def infer_ja_vlm_bench_wild(dataset,model,processor):
    results = []
    for idx, item in enumerate(dataset):
        image = item["image"].convert("RGB")
        question = item["question"]
        gt_answer = item.get("answer", None)

        print(f"\n====== [VQA] Entry {idx} - 質問処理開始 ======")

        result = generate_vqa_answer(image, question, gt_answer)
        results.append(result)

        print(f"[Entry {idx}] 質問: {question}")
        print(f"           正解      : {gt_answer}")
        print(f"           生成回答  : {result['pred_answer']}\n")

    return results

## 評価処理（ROUGE）
* 推論結果と正解のペアから、自然言語生成の類似度指標ROUGE-Lを計算。

* 文全体の一致度ではなく、部分一致（LCS）による評価が可能。

In [13]:
def evaluate_metric(results, bench_name):
    rouge = evaluate.load("rouge")
    valid_pairs = [
        (r["pred_answer"], r["gt_answer"])
        for r in results
        if r["gt_answer"] and r["pred_answer"] != "ERROR"
    ]

    if not valid_pairs:
        print(f"[Warning] {bench_name} に評価可能な結果がありません。")
        return

    predictions, references = zip(*valid_pairs)

    print(f"\n Evaluation for {bench_name} (ROUGE-L)")

    score = rouge.compute(
        predictions=predictions,
        references=references,
        rouge_types=["rougeL"]
    )

    print(f" - Rouge-L Score : {score['rougeL']:.4f}")

# 評価開始
## JA-VG-VQA-500

In [14]:
vg500 = load_dataset("SakanaAI/JA-VG-VQA-500")["test"].shuffle(seed=42).select(range(5))

README.md:   0%|          | 0.00/3.28k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/72.9M [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/139M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [15]:
vg_results = infer_ja_vg_vqa500(vg500,gemma_model,gemma_processor)
print_gpu_memory("after inference")


====== [VQA] Image 0 の質問処理 ======


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:653: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `64` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


[Image 0 | QA 0] 質問: ジョッキはいくつありますか？
           正解      : １つ
           生成回答  : ジョッキは1つです。

[Image 0 | QA 1] 質問: ジョッキの中に液体はど大体れくらい入っていますか？
           正解      : 上から１cmぐらいまでいっぱい
           生成回答  : ジョッキの中には、およそ330ml程度のビールが入っていると思います。

[Image 0 | QA 2] 質問: ジョッキの中の液体は何色ですか？
           正解      : 黄色
           生成回答  : ジョッキの中の液体は黄色です。

[Image 0 | QA 3] 質問: ジョッキの取っ手はどこ側についていますか？
           正解      : 右側
           生成回答  : ジョッキの取っ手は右側についています。

[Image 0 | QA 4] 質問: ジョッキはどこにありますか？
           正解      : 手前右端
           生成回答  : ジョッキはテーブルの右側にあります。

[Image 0 | QA 5] 質問: ジョッキの左にあるトレーは何色ですか？
           正解      : 赤色
           生成回答  : 赤いトレーです。

[Image 0 | QA 6] 質問: ジョッキの左にあるトレーの上に食べ物はいくつありますか？
           正解      : ２つ
           生成回答  : 2つ

[Image 0 | QA 7] 質問: 紙幣はどこにありますか？
           正解      : ジョッキの左にあるトレーの左斜め奥
           生成回答  : 紙幣は、テーブルの上の赤いプラスチックのトレイの中にあります。


====== [VQA] Image 1 の質問処理 ======
[Image 1 | QA 0] 質問: どこですか
           正解      : 外です
           生成回答  : インドの都市です。

[Image 1 | QA 1] 質問: お天気は
    

In [16]:
evaluate_metric(vg_results, "JA-VG-VQA-500")


 Evaluation for JA-VG-VQA-500 (ROUGE-L)
 - Rouge-L Score : 0.0500


## JA-VLM-Bench-In-the-Wild

In [17]:
vlm = load_dataset("SakanaAI/JA-VLM-Bench-In-the-Wild")["test"].shuffle(seed=42).select(range(10))

README.md:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/5.84M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/50 [00:00<?, ? examples/s]

In [18]:
vlm_results = infer_ja_vlm_bench_wild(vlm,gemma_model,gemma_processor)
print_gpu_memory("after inference")


====== [VQA] Entry 0 - 質問処理開始 ======
[Entry 0] 質問: この駅は何線のものですか？
           正解      : 画像から正確な路線名は読み取れませんが、京都へ行く電車のある駅であることが分かります。
           生成回答  : この駅は4線の駅です。


====== [VQA] Entry 1 - 質問処理開始 ======
[Entry 1] 質問: この画像に映っている魚の形をした飾りは、日本では何と呼ばれていますか？
           正解      : これらは「こいのぼり」と呼ばれています。
           生成回答  : これらの飾りは、ちゅうし（中日）と呼ばれています。


====== [VQA] Entry 2 - 質問処理開始 ======
[Entry 2] 質問: この画像に写っているタワーの名前は何ですか？
           正解      : 通天閣です。
           生成回答  : 金沢空中回廊


====== [VQA] Entry 3 - 質問処理開始 ======
[Entry 3] 質問: この画像にはどんな機械が写っていますか？
           正解      : 画像にはガチャガチャと呼ばれるカプセルトイの自動販売機が写っています。
           生成回答  : 複数の自動販売機が並んでいます。


====== [VQA] Entry 4 - 質問処理開始 ======
[Entry 4] 質問: この写真にはどのようなものが木についていますか？
           正解      : テントウムシの形をしたオブジェクトが木についています。
           生成回答  : 木には、赤いボディと黒いスポットが特徴の瓢虫の形をした、金属製の装飾品が取り付けられています。


====== [VQA] Entry 5 - 質問処理開始 ======
[Entry 5] 質問: この画像にはどんな標識が写っていますか？
           正解      : 画像には「進入禁止」の標識と、左矢印がある「一方通行」の標識が写っています。下部には「自転車を除く」という補助標識もあります。
           生

In [19]:
evaluate_metric(vlm_results, "JA-VLM-Bench-In-the-Wild")


 Evaluation for JA-VLM-Bench-In-the-Wild (ROUGE-L)
 - Rouge-L Score : 0.1667


## 学習用前処理
* LoRAにより効率的にファインチューニングするため、**プロンプト＋回答（ラベル）**の形式で入力を組み立てる。

* -100 でマスクされた部分は loss計算から除外される。

In [20]:
from PIL import Image
from io import BytesIO
import requests
def preprocess(example):
    image_url = example["url"]
    image = Image.open(BytesIO(requests.get(image_url).content)).convert("RGB")
    image = image.resize((224, 224))

    prompt = (
      f"<start_of_image><end_of_image>\n"
      f"この画像の以下の質問に対して簡潔に日本語で答えてください。\n"
      f"質問：{example['question']}\n"
      f"答え："
    )
    answer = example["category"]

    full_text = prompt + answer

    print("📌 prompt:")
    print(prompt)

    model_inputs = processor(
        text=full_text,
        images=image,
        return_tensors="pt",
        padding="max_length",
        truncation=False,
    )

    print("📌 tokenized input_ids:")
    print(model_inputs["input_ids"][0])
    print("📌 decoded input_ids:")
    print(tokenizer.decode(model_inputs["input_ids"][0]))

    labels = model_inputs["input_ids"].clone()

    special_token_ids = [
        tokenizer.pad_token_id,
        tokenizer.convert_tokens_to_ids("<start_of_image>"),
        tokenizer.convert_tokens_to_ids("<end_of_image>"),
        262144
    ]

    for token_id in special_token_ids:
        if token_id is not None:
            labels[labels == token_id] = -100

    print("📌 decoded labels (no -100):")
    print(tokenizer.decode(labels[0][labels[0] != -100]))

    return {
        "input_ids": model_inputs["input_ids"].squeeze(0),
        "attention_mask": model_inputs["attention_mask"].squeeze(0),
        "pixel_values": model_inputs["pixel_values"].squeeze(0),
        "labels": labels.squeeze(0)
    }

## LoRAファインチューニング設定

In [22]:
tokenizer = AutoTokenizer.from_pretrained("google/gemma-3-4b-it")
tokenizer.pad_token = tokenizer.eos_token
processor = AutoProcessor.from_pretrained("google/gemma-3-4b-it")
dataset = load_dataset("line-corporation/JIC-VQA", split="train").shuffle(seed=42).select(range(50))
train_dataset = dataset.map(preprocess, batched=False)
print_gpu_memory("after setting finetuning")

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.


📌 prompt:
<start_of_image><end_of_image>
この画像の以下の質問に対して簡潔に日本語で答えてください。
質問：この画像にはどのランドマークが映っていますか？次の四つの選択肢から正しいものを選んでください: ['金閣寺', '清水寺', '伏見稲荷大社', '姫路城']
答え：
📌 tokenized input_ids:
tensor([     2,    108, 255999, 262144, 262144, 262144, 262144, 262144, 262144,
        262144, 262144, 262144, 262144, 262144, 262144, 262144, 262144, 262144,
        262144, 262144, 262144, 262144, 262144, 262144, 262144, 262144, 262144,
        262144, 262144, 262144, 262144, 262144, 262144, 262144, 262144, 262144,
        262144, 262144, 262144, 262144, 262144, 262144, 262144, 262144, 262144,
        262144, 262144, 262144, 262144, 262144, 262144, 262144, 262144, 262144,
        262144, 262144, 262144, 262144, 262144, 262144, 262144, 262144, 262144,
        262144, 262144, 262144, 262144, 262144, 262144, 262144, 262144, 262144,
        262144, 262144, 262144, 262144, 262144, 262144, 262144, 262144, 262144,
        262144, 262144, 262144, 262144, 262144, 262144, 262144, 262144, 262144,
        262144, 262

* PEFT (LoRA) を用いて、事前学習モデルの一部（q_proj, v_proj）のみを効率よく更新。

* prepare_model_for_kbit_training() は量子化モデルでも学習できるように整える。

In [23]:
model = AutoModelForImageTextToText.from_pretrained(gemma_model_id, quantization_config=bnb_config, device_map="auto")
model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model_finetuning = get_peft_model(model, lora_config)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

    # Processorで画像＋テキストを同時に処理（最大長を制限して軽量化）
    inputs = gemma_processor(
        text=prompt,
        images=image,
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=256,  # 通常の512より短くすることで省メモリ化
    )

    # ラベル（答え）も軽量に処理
    labels = tokenizer(
        answer,
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=256  # カテゴリは短文なのでこれで十分
    )

    return {
        "input_ids": inputs["input_ids"].squeeze(0),
        "attention_mask": inputs["attention_mask"].squeeze(0),
        "pixel_values": inputs["pixel_values"].squeeze(0),
        "labels": labels["input_ids"].squeeze(0)
    }

In [24]:
class VQATrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        pixel_values = inputs.pop("pixel_values", None)
        labels = inputs.pop("labels", None)
        outputs = model(
            pixel_values=pixel_values,
            labels=labels,
            **inputs
        )
        loss = outputs.loss
        return (loss, outputs) if return_outputs else loss

training_args = TrainingArguments(
    output_dir="./gemma-lora",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    logging_steps=10,
    save_steps=2,
    save_total_limit=1,
    fp16=True,
    report_to="none"
)

trainer = VQATrainer(
    model=model_finetuning,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=default_data_collator,
)

trainer.train()
print_gpu_memory("after training")

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
It is strongly recommended to train Gemma3 models with the `eager` attention implementation instead of

Step,Training Loss
10,11.370400
20,9.498600
30,8.275000


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentran

[after training] GPU Memory Usage:
  Allocated: 7.41 GB
  Reserved : 11.45 GB


## モデルの再評価
* 学習後のモデルで再度推論・評価し、精度向上を比較する。


In [25]:
finetuned_vqa500_results = infer_ja_vg_vqa500(
    dataset=vg500,
    model=trainer.model,
    processor=gemma_processor
)
evaluate_metric(finetuned_vqa500_results, "JA-VG-VQA-500")
print_gpu_memory("after inference")


====== [VQA] Image 0 の質問処理 ======


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:653: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `64` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


[Image 0 | QA 0] 質問: ジョッキはいくつありますか？
           正解      : １つ
           生成回答  : ジョッキは1つです。

[Image 0 | QA 1] 質問: ジョッキの中に液体はど大体れくらい入っていますか？
           正解      : 上から１cmぐらいまでいっぱい
           生成回答  : ジョッキの中には、およそ330ml程度のビールが入っていると思います。

[Image 0 | QA 2] 質問: ジョッキの中の液体は何色ですか？
           正解      : 黄色
           生成回答  : ジョッキの中の液体は黄色です。

[Image 0 | QA 3] 質問: ジョッキの取っ手はどこ側についていますか？
           正解      : 右側
           生成回答  : ジョッキの取っ手は右側についています。

[Image 0 | QA 4] 質問: ジョッキはどこにありますか？
           正解      : 手前右端
           生成回答  : ジョッキはテーブルの右側にあります。

[Image 0 | QA 5] 質問: ジョッキの左にあるトレーは何色ですか？
           正解      : 赤色
           生成回答  : 赤いトレーです。

[Image 0 | QA 6] 質問: ジョッキの左にあるトレーの上に食べ物はいくつありますか？
           正解      : ２つ
           生成回答  : 2つ

[Image 0 | QA 7] 質問: 紙幣はどこにありますか？
           正解      : ジョッキの左にあるトレーの左斜め奥
           生成回答  : 紙幣は、テーブルの上の赤いプラスチックのトレイの中にあります。


====== [VQA] Image 1 の質問処理 ======
[Image 1 | QA 0] 質問: どこですか
           正解      : 外です
           生成回答  : インドの都市です。

[Image 1 | QA 1] 質問: お天気は
    

In [26]:
finetuned_wild_results = infer_ja_vlm_bench_wild(
    dataset=vlm,
    model=trainer.model,
    processor=gemma_processor
)
evaluate_metric(finetuned_wild_results, "JA-VLM-Bench-In-the-Wild")
print_gpu_memory("after inference")


====== [VQA] Entry 0 - 質問処理開始 ======
[Entry 0] 質問: この駅は何線のものですか？
           正解      : 画像から正確な路線名は読み取れませんが、京都へ行く電車のある駅であることが分かります。
           生成回答  : この駅は4線の駅です。


====== [VQA] Entry 1 - 質問処理開始 ======
[Entry 1] 質問: この画像に映っている魚の形をした飾りは、日本では何と呼ばれていますか？
           正解      : これらは「こいのぼり」と呼ばれています。
           生成回答  : これらの飾りは、ちゅうし（中日）と呼ばれています。


====== [VQA] Entry 2 - 質問処理開始 ======
[Entry 2] 質問: この画像に写っているタワーの名前は何ですか？
           正解      : 通天閣です。
           生成回答  : 金沢空中回廊


====== [VQA] Entry 3 - 質問処理開始 ======
[Entry 3] 質問: この画像にはどんな機械が写っていますか？
           正解      : 画像にはガチャガチャと呼ばれるカプセルトイの自動販売機が写っています。
           生成回答  : 複数の自動販売機が並んでいます。


====== [VQA] Entry 4 - 質問処理開始 ======
[Entry 4] 質問: この写真にはどのようなものが木についていますか？
           正解      : テントウムシの形をしたオブジェクトが木についています。
           生成回答  : 木には、赤いボディと黒いスポットが特徴の瓢虫の形をした、金属製の装飾品が取り付けられています。


====== [VQA] Entry 5 - 質問処理開始 ======
[Entry 5] 質問: この画像にはどんな標識が写っていますか？
           正解      : 画像には「進入禁止」の標識と、左矢印がある「一方通行」の標識が写っています。下部には「自転車を除く」という補助標識もあります。
           生

## 苦戦した点

本プロジェクトを進める中で、いくつかの技術的・運用的な課題に直面し、解決に時間を要した。以下に主な苦戦点を整理する。(4/13時点での)

1. 画像とテキストを組み合わせてモデルに入力し、推論を行う実装の構築に時間がかかった。
マルチモーダルな入力を正しく処理するためのプロンプト設計や、前処理（画像のRGB変換など）に習熟する必要があり、試行錯誤を繰り返した。

2. ベンチマーク評価の方法を調査するのに時間を要した。
特にROUGEスコアなどの自動評価指標の扱い方や、どのような条件で評価が行われるのかを理解するまでに苦労した。

3. ベンチマーク評価の結果に違和感があり、コードや処理ロジックの確認、さらにはGPUメモリの制限による処理失敗への対応に時間を要した。
データセットの件数が多くなるとGPUメモリを超過するため、バッチサイズや処理単位の調整を検討する必要があった。

4. Heron-Benchについて調査した際、GPT-4を用いた評価プロセス（生成された2つの回答を比較し、GPT-4が点数を付与する方式）の理解に時間がかかった。
さらに、GPT-4の使用にはコストがかかるため、予算の計算やAPI利用準備などを含めた検討も必要となった。結果的に、この評価方法は後回しとし、今回は実施を見送った。

5. VLM（Vision-Language Model）のファインチューニングを可能にする実装方法の調査にも多くの時間を要した。
特に、マルチモーダル入力に対応するモデル構造の改変や、LoRAなどの軽量ファインチューニング手法の適用に多くの技術的課題が存在し、実装が思うように進まなかった。

さらに4/13以降でこの課題に取り組んだ中で、苦戦した点について整理する。

1. GPUの使用メモリ上限（15GB）を超えないようにするため、量子化（4bit）やLoRAの導入などの工夫が必要だった。既存のコードを細かく修正・調整する必要があり、その作業に多くの時間を要した。

2. ファインチューニングが正しく行えているかを確認するために、学習や再評価のコードの修正と再実行を何度か試みた。しかし、一度の学習や推論でもGPUメモリの消費が大きいため、複数回の実験を継続することが難しい状況であった。日によってはGPUの上限に早々に達してしまい、作業が思うように進まないこともあった。

3. ファインチューニングの学習ログ上は、損失関数の減少が確認できており、学習自体は進んでいるように見えた。しかし、その後の再評価（JA-VG-VQA-500やJA-VLM-Bench-In-the-Wild）において、生成される回答が学習前と全く同じであるという現象に直面した。この点については、学習が反映されていない、または反映されていても出力に影響していない可能性が考えられるものの、明確な原因はまだ特定できておらず、今後の重要な課題として残っている。





## 参考資料：

*   https://huggingface.co/google/gemma-3-4b-it
*   https://huggingface.co/datasets/SakanaAI/JA-VLM-Bench-In-the-Wild
*   https://huggingface.co/datasets/SakanaAI/JA-VG-VQA-500
*   https://huggingface.co/datasets/turing-motors/Japanese-Heron-Bench
*   https://huggingface.co/datasets/line-corporation/JIC-VQA
*   https://qiita.com/tinymouse/items/1c5d49579e95369b1d0b
*   https://note.com/owlet_notes/n/na7416b0414b7
*   https://zenn.dev/watamoo/articles/0590ad46d7b26b
